# **Part 1: Take a Look at Cartpole Rl Agent**

1. According to the tutorials, if we want to edit the environment configuration, action space, observation space, reward function, or termination condition of the Isaac-Cartpole-v0 task, which file should we look at, and where is each part located?

2. What are the action space and observation space for an agent defined in the Isaac-Cartpole-v0 task?

3. How can episodes in the Isaac-Cartpole-v0 task be terminated?

4. How many reward terms are used to train an agent in the Isaac-Cartpole-v0 task?

# **Part 2: Playing with Cartpole Rl Agent**

Let us adjust the weight of each reward term specified in the Isaac-Cartpole-v0 task and train the agent. Which results will be affected by this adjustment, and why? Submit the answers.

You may further explore by modifying other aspects, such as the agent's action space, observation space, and termination conditions.

# **Part 3: Mapping RL Fundamentals**

1. What is reinforcement learning and its components according to your understanding? Giving examples of each component according to the diagram consider the Cartpole problem.

2. What is the difference between the reward, return, and the value function?

3. Consider policy, state, value function, and model as mathematical functions, what would each one take as input and output?